In [1]:
import regex as re
from tqdm import tqdm
from scipy import sparse
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
%matplotlib inline

## Часть 1. Обучение модели word2vec [40 баллов]
1. Объедините положительные и отрицательные твиты в один массив
2. Разбейте всю коллекцию отзывов на предложения. Лемматизируйте все слова.
3. Обучите по коллекции предложений word2vec
4. Приведите несколько удачных и неудачных примеров решения стандартных тестов для word2vec:
    - тест на определение ближайших слов
    - тест на аналогии (мужчина – король : женщина – королева)
    - тест на определение лишнего слова.
5. Постройте визуализацию TSNE для топ-100 (или топ-500) слов и найдите осмысленные кластеры слов

Ссылка на примеры визуализаций: https://towardsdatascience.com/game-of-thrones-word-embeddings-does-r-l-j-part-2-30290b1c0b4b

In [2]:
columns = ['id', 'date', 'name', 'text', 'type', 'rep', 'rtw', 'fav', 'stcount', 
            'fol', 'frien', 'listcount']

In [3]:
positive = pd.read_csv('Data/positive.csv', sep=';',  names=columns)
negative = pd.read_csv('Data/negative.csv', sep=';',  names=columns)

In [4]:
positive['is_positive'] = 1
negative['is_positive'] = 0

In [5]:
negative.head(3)

,id,date,name,text,type,rep,rtw,fav,stcount,fol,frien,listcount,is_positive
0,408906762813579264,1386325944,dugarchikbellko,на работе был полный пиддес :| и так каждое за...,-1,0,0,0,8064,111,94,2,0
1,408906818262687744,1386325957,nugemycejela,"Коллеги сидят рубятся в Urban terror, а я из-з...",-1,0,0,0,26,42,39,0,0
2,408906858515398656,1386325966,4post21,@elina_4post как говорят обещаного три года жд...,-1,0,0,0,718,49,249,0,0


In [6]:
df = (positive.append(negative)).reset_index(drop=True)

In [7]:
positive.shape, negative.shape, df.shape

((114911, 13), (111923, 13), (226834, 13))

In [8]:
df = pd.DataFrame(df[['text', 'is_positive']])

In [9]:
# word = re.compile('[А-Яа-я]+|\.+|\!+|\?+|\)+')
word = re.compile('[А-Яа-я]+')
df['text'] = df['text'].apply(lambda x: ' '.join(re.findall(word, x.lower())))
# symbol = re.compile('\.+|\!+|\?+|\)+') # окончания предложений
# df['ttext'] = df['ttext'].apply(lambda x: re.sub(symbol, '.', x))

In [10]:
df.head()

,text,is_positive
0,хоть я и школота но поверь у нас то же самое о...,1
1,да все таки он немного похож на него но мой ма...,1
2,ну ты идиотка я испугалась за тебя,1
3,кто то в углу сидит и погибает от голода а мы ...,1
4,вот что значит страшилка но блин посмотрев все...,1


In [11]:
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from pymorphy2 import MorphAnalyzer

In [12]:
stopw = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также']
def  stop_words(text, stopwords = stopw):
    try:
        return " ".join([token for token in text.split() if not token in stopwords and len(token) > 2])
    except:
        return ""

In [13]:
pm2 = MorphAnalyzer()

def  remove_lemm(text):
    try:
        return " ".join([pm2.parse(token)[0].normal_form for token in text.split()])
    except:
        return ""

In [14]:
def preprocessing(text):
    no_stopwords = stop_words(text)
    lemmas = remove_lemm(no_stopwords)
    return lemmas

In [15]:
df['text'] = df['text'].apply(lambda x: preprocessing(x))

In [16]:
df['text'] = df['text'].apply(lambda x: x.split(' '))

In [17]:
df.head()

,text,is_positive
0,"[школотый, поверь, самый, общество, профилиров...",1
1,"[таки, немного, похожий, мальчик, равно]",1
2,"[идиотка, испугаться]",1
3,"[угол, сидеть, погибать, голод, порция, взять,...",1
4,"[значит, страшилка, блин, посмотреть, часть, с...",1


In [53]:
model = Word2Vec(df.text, size=100, window=3, min_count=20, workers=4)

In [54]:
model.save("word2v.model")

In [18]:
model = Word2Vec.load("word2v.model")

### Теперь посмотрим ближайшие слова

In [57]:
model.most_similar("сайт")[:3]

C:\My Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('приложение', 0.9386524558067322),
 ('блог', 0.9179162383079529),
 ('ссылка', 0.9177023768424988)]

In [58]:
model.most_similar("голод")[:3]

C:\My Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('капельница', 0.9647878408432007),
 ('затем', 0.9569962024688721),
 ('стоя', 0.9561183452606201)]

In [59]:
model.most_similar("блин")[:3]

C:\My Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('блиина', 0.7782334685325623),
 ('ааа', 0.7753502130508423),
 ('тож', 0.7673264741897583)]

### Проверим аналоги

In [89]:
model.most_similar(positive=["жара", "зима"], negative=["лето"])[:3]

C:\My Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('вод', 0.8576784133911133),
 ('холод', 0.8381669521331787),
 ('лёд', 0.8151445388793945)]

In [91]:
model.most_similar(positive=["сша", "путин"], negative=["россия"])[:3]

C:\My Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('синдромканделаки', 0.8908705711364746),
 ('журналист', 0.8765182495117188),
 ('ресурс', 0.8643521070480347)]

### Найдем лишнее

In [95]:
model.doesnt_match("чатиться писать общаться радуга".split())

C:\My Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.


'радуга'

In [92]:
model.doesnt_match("планета космос байконур париж".split())

C:\My Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `doesnt_match` (Method will be removed in 4.0.0, use self.wv.doesnt_match() instead).
  """Entry point for launching an IPython kernel.
C:\My Programs\Anaconda3\lib\site-packages\gensim\models\keyedvectors.py:877: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)


'планета'

Визуализируем полученное пространство векторов

In [128]:
from nltk import FreqDist

all_words = FreqDist()
for word in df['text']:
    all_words.update(word)

In [146]:
top_words = []

for i in all_words.most_common(300):
    top_words.append(i[0])

In [147]:
top_words = [w for w in top_words if len(w) > 4]
print (top_words[:10])

['хотеть', 'такой', 'сегодня', 'очень', 'просто', 'человек', 'знать', 'любить', 'завтра', 'самый']


In [148]:
top_words_vec = model[top_words]

C:\My Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [34]:
from sklearn.manifold import TSNE

In [36]:
tsne = TSNE()
top_words_tsne = tsne.fit_transform(top_words_vec)

In [151]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

Если увеличить и рассмотреть 'сгустки' векторов, то они в основном на похожие тематики. Например: в верхнем левом углу: минута, неделя, месяц

## Часть 2. Анализ тональности [60 баллов]

Отделите 25% положительных и 25% отрицательных твитов как тестовую выборку. Обучите классификатор, отличающий положительно окрашенные твиты от отрицательно окрашенных на обучающей выборке (оставшиеся 75%) и оцените их accuracy на выборке тестовой. А лучше несколько классификаторов, устроенных по-разному. Если у вас получилось несколько очень похожих классификаторов (скажем, один и тот же класс из sklearn с разными гиперпараметрами), они будут считаться не разными методами, а разными версиями одного метода и учитываться будет только лучшая версия. Тем не менее, по возможности не удаляйте их из ноутбука! Очень интересно, как вы пришли к результату, к которому вы пришли.

Можно использовать предобученные модели. К обучающей выборке можно добавить какие-нибудь ещё данные, которые вы сами где-то найдёте, а можно наоборот - вообще её не использовать (сделать классификатор, основанный на вручную написанных правилах).

Если у вас будет хотя бы один работающий классификатор, ваша оценка за часть 2 будет вычислена по формуле:

$$
\min( 10 + \sum_i (a_i - 50); 60 )
$$

где $a_i$ - accuracy одного из ваших (разных!) методов на тестовой выборке.

В этом задании (при желании) можно применить все знания, полученные за курс:
- Морфология и синтаксический парсинг
- Тематические модели
- Bag of words и TF/IDF
- Дистрибутивная семантика
- Классические классификаторы (RandomForest)
- Нейронные классификаторы
- Языковые модели
- Attention
- Извлечение терминов

А так же задавать в слаке любые вопросы

Как видно из формулы, у вас есть выбор - брать количеством или качеством :)

Удачи и да пребудет с вами BERT.

In [182]:
def sent_to_vec(sent, w2v = model):
    vec = np.array([w2v[w] for w in sent if w in w2v])
    if len(vec):
        return(np.mean(vec, axis=0))
    else:
        return(np.zeros(100))

In [183]:
df['mean_vec'] = df['text'].apply(lambda x: sent_to_vec(x))

C:\My Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  
C:\My Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [184]:
df.head()

,text,is_positive,mean_vec
0,"[школотый, поверь, самый, общество, профилиров...",1,"[0.21145439, -0.52777493, -0.2547268, 0.464470..."
1,"[таки, немного, похожий, мальчик, равно]",1,"[0.0074229212, -0.14906716, -0.09295411, -0.29..."
2,"[идиотка, испугаться]",1,"[-0.17222708, -0.1628049, -0.15499324, 0.09118..."
3,"[угол, сидеть, погибать, голод, порция, взять,...",1,"[-0.22343801, 0.098665655, -0.14157748, 0.3297..."
4,"[значит, страшилка, блин, посмотреть, часть, с...",1,"[-0.13393934, -0.20025873, -0.06487245, 0.0321..."


In [15]:
from sklearn.model_selection import StratifiedShuffleSplit

In [188]:
splitter = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25, random_state = 13)

for train_index, test_index in splitter.split(df, df['is_positive']):
    X_train = df.iloc[train_index]
    X_test = df.iloc[test_index]
    y_train = df['is_positive'].iloc[train_index]
    y_test = df['is_positive'].iloc[test_index]

In [208]:
X_train = X_train['mean_vec'].tolist()
X_test = X_test['mean_vec'].tolist()

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [211]:
rf = RandomForestClassifier(n_jobs=-1)

max_depth = [5, 7, 9, 12, 15]
min_samples_leaf = [5, 7, 9, 12, 15]
grid_params = {'max_depth':max_depth, 'min_samples_leaf':min_samples_leaf}
grid_rf = GridSearchCV(rf, grid_params, cv=5, n_jobs=-1)

In [212]:
grid_rf.fit(X_train, y_train)

C:\My Programs\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
C:\My Programs\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators='warn', n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'max_depth': [5, 7, 9, 12, 15], 'min_samples_leaf': [5, 7, 9, 12, 15]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [213]:
y_pred = grid_rf.best_estimator_.predict(X_test)

In [216]:
accuracy_score(y_test, y_pred)

0.6387698601632897

посмотрим как линейный алгоритм справится с этой задачей

In [217]:
from sklearn.linear_model import LogisticRegression 
log = LogisticRegression(n_jobs=-1)

grid_params = {'C': [0.01, 0.05, 0.1, 0.5, 1, 1.5]}
grid_log = GridSearchCV(log, grid_params, cv=5, n_jobs=-1)

In [218]:
grid_log.fit(X_train, y_train)

C:\My Programs\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\My Programs\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:1300: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = 4.
  " = {}.".format(effective_n_jobs(self.n_jobs)))


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn', n_jobs=-1,
          penalty='l2', random_state=None, solver='warn', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': [0.01, 0.05, 0.1, 0.5, 1, 1.5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [219]:
y_pred = grid_log.best_estimator_.predict(X_test)
accuracy_score(y_test, y_pred)

0.621224144315717

### взвесим наши вектора с помощью tf-idf

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [20]:
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 30)
df_tfidf = tfidf.fit_transform([' '.join(tokens) for tokens in df['text']])

In [21]:
table = df_tfidf.toarray()

In [22]:
table[0][table[0] != 0]

array([0.48812934, 0.53727708, 0.47548669, 0.2908606 , 0.40295856])

In [25]:
df = df.reset_index()
df.columns = ['ind', 'text', 'is_positive']

In [159]:
def sent_to_vec_with_tfidf(sent, num, w2v = model):
    list_names = tfidf.get_feature_names()
    vec_tfidf = []
    for w in sent:
        if w in w2v:
            try:
                ind = list_names.index(w)
                coef = table[num][ind]
                vec_tfidf.append(w2v[w] * coef)
            except:
                vec_tfidf.append(np.zeros(100))
    vec = np.array(vec_tfidf)
    if len(vec):
        return (np.mean(vec, axis=0))
    else:
        return(np.zeros(100))

In [160]:
df['mean_vec_tfidf'] = df.apply(lambda x: sent_to_vec_with_tfidf(x.text, x.ind), axis=1)

C:\My Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
C:\My Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if __name__ == '__main__':


In [164]:
splitter = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25, random_state = 13)

for train_index, test_index in splitter.split(df, df['is_positive']):
    X_train = df.iloc[train_index]
    X_test = df.iloc[test_index]
    y_train = df['is_positive'].iloc[train_index]
    y_test = df['is_positive'].iloc[test_index]

In [165]:
X_train = X_train['mean_vec_tfidf'].tolist()
X_test = X_test['mean_vec_tfidf'].tolist()

In [180]:
rf = RandomForestClassifier(n_jobs=-1, max_depth=20, min_samples_leaf=15)

In [182]:
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

C:\My Programs\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [183]:
accuracy_score(y_test, y_pred)

0.6335502301222028

получили практически такой же результат

## теперь попробуем другой embedding

In [16]:
from bpemb import BPEmb

In [17]:
from pathlib import WindowsPath
bpemb_ru = BPEmb(lang="ru", dim=100, cache_dir = WindowsPath('C:\My Programs'))

In [18]:
bpemb_ru_2 = BPEmb(lang="ru", dim=50, cache_dir = WindowsPath('C:\My Programs'))

In [19]:
df = (positive.append(negative)).reset_index(drop=True)

In [20]:
df = pd.DataFrame(df[['text', 'is_positive']])

In [21]:
word = re.compile('[А-Яа-я]+')
df['text'] = df['text'].apply(lambda x: ' '.join(re.findall(word, x.lower())))

In [22]:
df.head()

,text,is_positive
0,хоть я и школота но поверь у нас то же самое о...,1
1,да все таки он немного похож на него но мой ма...,1
2,ну ты идиотка я испугалась за тебя,1
3,кто то в углу сидит и погибает от голода а мы ...,1
4,вот что значит страшилка но блин посмотрев все...,1


In [23]:
stopw = stopwords.words('russian') + ['это', 'наш' , 'тыс', 'млн', 'млрд', 'также']
def  stop_words(text, stopwords = stopw):
    try:
        return " ".join([token for token in text.split() if not token in stopwords and len(token) > 2])
    except:
        return ""

здесь лематизацию делать не будем

In [24]:
df['text'] = df['text'].apply(lambda x: stop_words(x))

In [23]:
def sent_to_bpe(sent):
    vec = bpemb_ru.embed(sent)
    if len(vec):
        return(np.mean(vec, axis=0))
    else:
        return(np.zeros(100))

In [24]:
df['text_bpe'] = df['text'].apply(lambda x: sent_to_bpe(x))

In [25]:
df.head()

,text,is_positive,text_bpe
0,школота поверь самое общество профилирующий пр...,1,"[0.014792068, -0.10914229, 0.094901, 0.0275122..."
1,таки немного похож мальчик равно,1,"[-0.337034, -0.063679755, 0.1094545, 0.0075524..."
2,идиотка испугалась,1,"[-0.17493764, 0.105021626, 0.22462825, 0.23207..."
3,углу сидит погибает голода порции взяли хотя ж...,1,"[-0.30528587, 0.10548739, 0.0524021, 0.1353126..."
4,значит страшилка блин посмотрев части создастс...,1,"[-0.19662201, -0.011566614, -0.03991857, 0.264..."


In [27]:
splitter = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25, random_state = 13)

for train_index, test_index in splitter.split(df, df['is_positive']):
    X_train = df.iloc[train_index]
    X_test = df.iloc[test_index]
    y_train = df['is_positive'].iloc[train_index]
    y_test = df['is_positive'].iloc[test_index]

In [28]:
X_train = X_train['text_bpe'].tolist()
X_test = X_test['text_bpe'].tolist()

In [29]:
rf = RandomForestClassifier(n_jobs=-1, max_depth=15, min_samples_leaf=15)

In [30]:
rf.fit(X_train, y_train)

C:\My Programs\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=15, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [31]:
y_pred = rf.predict(X_test)

In [32]:
accuracy_score(y_test, y_pred)

0.605018603748964

Получили результат немного хуже чем с word2vec. Теперь проверим какой результат получит нейросеть

In [50]:
from keras.layers import Dense, Activation, BatchNormalization
from keras.layers import Input
from keras.activations import tanh, sigmoid
from keras.models import Model
import keras

In [85]:
X_train = np.array(X_train)
X_test = np.array(X_test)

In [72]:
X_train.shape

(170125, 100)

In [74]:
nn_input = Input(shape=(100,))
x = Dense(256, activation=tanh)(nn_input)
x = BatchNormalization()(x)
x = Dense(256, activation=tanh)(x)
x = BatchNormalization()(x)
x = Dense(128, activation=tanh)(x)
x = BatchNormalization()(x)
output = Dense(1, activation=sigmoid)(x)

model = Model(inputs=[nn_input], output=[output] )

C:\My Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=[<tf.Tenso..., outputs=[<tf.Tenso...)`
  # Remove the CWD from sys.path while we load stuff.


In [119]:
model.evaluate

optimizer = keras.optimizers.Adam(lr=4e-3)
model.compile(optimizer=optimizer, 
              loss='binary_crossentropy',  # функция потерь binary_crossentropy (log loss)
              metrics=['accuracy'])

In [120]:
model.fit(X_train, y_train, batch_size=200000, epochs=100)

Epoch 1/100
170125/170125 [==============================] - 5s 31us/step - loss: 0.5717 - acc: 0.6967
Epoch 2/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.6243 - acc: 0.6438
Epoch 3/100
170125/170125 [==============================] - 3s 19us/step - loss: 0.5753 - acc: 0.6934
Epoch 4/100
170125/170125 [==============================] - 3s 19us/step - loss: 0.6007 - acc: 0.6675
Epoch 5/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5947 - acc: 0.6718
Epoch 6/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5782 - acc: 0.6899
Epoch 7/100
170125/170125 [==============================] - 3s 19us/step - loss: 0.5785 - acc: 0.6882
Epoch 8/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5851 - acc: 0.6795
Epoch 9/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5830 - acc: 0.6833
Epoch 10/100
170125/170125 [==============================] - 3s 18us/ste

Epoch 80/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5406 - acc: 0.7230
Epoch 81/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5400 - acc: 0.7238
Epoch 82/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5394 - acc: 0.7243
Epoch 83/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5388 - acc: 0.7247
Epoch 84/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5381 - acc: 0.7251
Epoch 85/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5375 - acc: 0.7257
Epoch 86/100
170125/170125 [==============================] - 3s 19us/step - loss: 0.5369 - acc: 0.7262
Epoch 87/100
170125/170125 [==============================] - 3s 18us/step - loss: 0.5363 - acc: 0.7268
Epoch 88/100
170125/170125 [==============================] - 3s 19us/step - loss: 0.5356 - acc: 0.7273
Epoch 89/100
170125/170125 [==============================] - 4s

In [121]:
y_pred = model.predict(X_test)

In [122]:
y_pred[y_pred<0.5] = int(0)
y_pred[y_pred>0.5] = int(1)

In [123]:
accuracy_score(y_test, y_pred)

0.601033345677053

Результат не очень хороший и похоже наша сеть немного переучилась. Теперь будем не усреднять эмбединги, а подадим их в рекурентную сеть

In [25]:
from torch.nn.modules import LSTM
import torch.nn as nn
import torch.optim as optim
import torch

In [27]:
df['text_bpe'] = df['text'].apply(lambda x: sent_to_bpe_2(x))

In [244]:
df['opt_num'] = df['text_bpe'].apply(lambda x: x.shape[0])
df.opt_num.describe(percentiles=[0.7, 0.8, 0.9, 0.95])

count    226834.000000
mean         14.117121
std           8.225647
min           1.000000
50%          13.000000
70%          17.000000
80%          20.000000
90%          24.000000
95%          27.000000
max         111.000000
Name: opt_num, dtype: float64

Видим что 25 "Bpemp"  описывает более 90% всех предложений. Это количесво последовательности и возьмем 

In [26]:
def sent_to_bpe_full(sent):
    vec = bpemb_ru.embed(sent)
    if len(vec):
        return(vec)
    else:
        return(np.zeros(100).reshape(1,-1))

In [27]:
df['text_bpe'] = df['text'].apply(lambda x: sent_to_bpe_full(x))

In [28]:
splitter = StratifiedShuffleSplit(n_splits = 1, test_size = 0.25, random_state = 13)

for train_index, test_index in splitter.split(df, df['is_positive']):
    X_train = df.iloc[train_index]
    X_test = df.iloc[test_index]
    y_train = df['is_positive'].iloc[train_index]
    y_test = df['is_positive'].iloc[test_index]

In [29]:
X_train = X_train['text_bpe'].tolist()
X_test = X_test['text_bpe'].tolist()

In [57]:
def GetValues(words):
    return bpemb_ru.encode_ids(words)
def GetWords(values):
    return bpemb_ru.decode_ids(values)

In [106]:
from torch.utils.data import Dataset, DataLoader

class BpeDataset(Dataset):
    def __init__(self):
        super().__init__()
        self.vocab_size = bpemb_ru.vocab_size
        self.data = X_train
        self.target = y_train
        self.eos = bpemb_ru.vectors[bpemb_ru.EOS].tolist()
        
    def __getitem__(self, index):
        x = self.data[index]
        y = self.target.iloc[index]
        x_list = x[1:].tolist()
        x_list.append(self.eos)
        y_for_seq2seq = np.array(x_list)
        return x, y     
        
    def __len__(self):
        return len(self.data)

In [107]:
trn_ds = BpeDataset()
trn_dl = DataLoader(trn_ds, shuffle=True)

In [108]:
class LSTMmodel(nn.Module):
    def __init__(self, input_size=100, hidden_size=50, num_layers=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers)
        self.out = nn.Linear(hidden_size, bpemb_ru.vocab_size)
        self.vocab_size = bpemb_ru.vocab_size
        self.softmax = nn.LogSoftmax(dim=1)
        self.classifier = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.Tanh(),
            nn.Dropout(),
            nn.Linear(256, 128),
            nn.Tanh(),
            nn.Dropout(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )
        self.hidden_size = hidden_size
        
    def forward(self, x, c_prev, h_prev):
        x = x.float()
        lstm_out, (c_prev, h_prev) = self.lstm(x, (c_prev, h_prev))
        tag_space = self.classifier(lstm_out.view(len(x), -1))
        return tag_space, (c_prev, h_prev)

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [109]:
hidden_size = 50
model = LSTMmodel()

In [110]:
def train_one_epoch(model, loss_fn, optimizer):
    model.train()
    for line_num, (x, y) in enumerate(trn_dl):
        optimizer.zero_grad()
        c_prev = torch.zeros([1, 1, hidden_size], dtype=torch.float, device='cpu')
        h_prev = torch.zeros_like(c_prev)
        x, y = x.to("cpu"), y.to("cpu")
        for i in range(x.shape[1]):
            y_pred, (c_prev, h_prev)  = model.forward(x[:, i].view(1, 1, -1), c_prev, h_prev)
            y_true = torch.tensor(y.view(1, -1)).float()
        loss = loss_fn(y_pred, y_true)
    
         
        if loss != 0:
            loss.backward()
            optimizer.step()
        if line_num % 3000 == 0:
            print (line_num)

In [111]:
def train(model, loss_fn, optimizer, epochs=1):
    for e in range(1, epochs+1):
        print('Epoch:{}'.format(e))
        train_one_epoch(model, loss_fn, optimizer)

In [112]:
loss_fn = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-2)

In [ ]:
train(model, loss_fn, optimizer, epochs = 20)

Epoch:1


C:\My Programs\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  # Remove the CWD from sys.path while we load stuff.


0
3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
48000
51000
54000
57000
60000
63000
66000
69000
72000
75000
78000
81000
84000
87000
90000
93000
96000
99000
102000
105000
108000
111000
114000
117000
120000
123000
126000
129000
132000
135000
138000
141000
144000
147000
150000
153000
156000
159000
162000
165000
168000
Epoch:2
0
3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
48000
51000
54000
57000
60000
63000
66000
69000
72000
75000
78000
81000
84000
87000
90000
93000
96000
99000
102000
105000
108000
111000
114000
117000
120000
123000
126000
129000
132000
135000
138000
141000
144000
147000
150000
153000
156000
159000
162000
165000
168000
Epoch:3
0
3000
6000
9000
12000
15000
18000
21000
24000
27000
30000
33000
36000
39000
42000
45000
48000
51000
54000
57000
60000
63000
66000
69000
72000
75000
78000
81000
84000
87000
90000
93000
96000
99000
102000
105000
108000
111000
114000
117000
120000
123000
126000
129000
1

Такой плохой результат получается вероятно по тому, что мы работаем с моделью обученной на Википедиа, что для анализа Твитера не очень подходит. И к тому же суммирование всех векторов тоже делать не очень правильно. Но дообучать её не будем, а пойдем дальше

In [1]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
# import logging

# logging.basicConfig(level=logging.INFO)

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenized input
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = 8
tokenized_text[masked_index] = '[MASK]'

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])

In [4]:
# Load pre-trained model (weights)
model = BertModel.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict hidden states features for each layer
with torch.no_grad():
    encoded_layers, _ = model(tokens_tensor, segments_tensors)
# We have a hidden states for each of the 12 layers in model bert-base-uncased
assert len(encoded_layers) == 12

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at C:\Users\Илья Курошев\.pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file C:\Users\Илья Курошев\.pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir C:\Users\8EA2~1\AppData\Local\Temp\tmppxt_46n2
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}



In [5]:
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model.eval()

# If you have a GPU, put everything on cuda
tokens_tensor = tokens_tensor.to('cuda')
segments_tensors = segments_tensors.to('cuda')
model.to('cuda')

# Predict all tokens
with torch.no_grad():
    predictions = model(tokens_tensor, segments_tensors)

# confirm we were able to predict 'henson'
predicted_index = torch.argmax(predictions[0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
assert predicted_token == 'henson'

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased.tar.gz from cache at C:\Users\Илья Курошев\.pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba
INFO:pytorch_pretrained_bert.modeling:extracting archive file C:\Users\Илья Курошев\.pytorch_pretrained_bert\9c41111e2de84547a463fd39217199738d1e3deb72d4fec4399e6e241983c6f0.ae3cef932725ca7a30cdcb93fc6e09150a55e2a130ec7af63975a16c153ae2ba to temp dir C:\Users\8EA2~1\AppData\Local\Temp\tmpsltiwiol
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

INFO:pytor

In [6]:
predicted_index

27227

In [8]:
predicted_token

'henson'